In [1]:
# general packages
import numpy as np
import pandas as pd
import datatable as dt
import gc

## 1. Data preposessing

In [2]:
%%time
#parse_dates = ['CreatedDate', 'RequestedDate', 'AccTime', 'PupTime', 'DelTime']
#A = pd.read_csv('Booking_Dispatch_training_set.csv', parse_dates = parse_dates, 
#date_parser = lambda x: pd.to_datetime(x).strftime("%Y-%m-%d %H:%M:%S"))
#data_parsed = A.to_csv('data_parsed.csv')

Wall time: 0 ns


In [3]:
%%time
#parse_dates = ['CreatedDate', 'RequestedDate']
#B = pd.read_csv('Booking_Dispatch_training_set.csv', parse_dates = parse_dates, 
#date_parser = lambda x: pd.to_datetime(x).strftime("%Y-%m-%d %H:%M:%S"))
#data_parsed = B.to_csv('val_data_parsed.csv')

Wall time: 0 ns


In [4]:
%%time
train_data_datatable = dt.fread('data_parsed.csv') 
data = train_data_datatable.to_pandas()
data = data.iloc[:,1:]

Wall time: 2min 17s


In [5]:
%%time
val_data_datatable = dt.fread('val_data_parsed.csv')
val_data = val_data_datatable.to_pandas()
val_data = val_data.iloc[:,1:]

Wall time: 2min 25s


In [6]:
data.shape

(3000000, 37)

In [7]:
val_data.shape

(3000000, 37)

In [8]:
#subtract the features and target variable for modeling 
train_features = data[['BookingSource','Conditions',
     'HasCondition','IsAccountBooking','CreatedDate','RequestedDate',
     'BookingFleet','DispatchFleet','AreaNumber','PuPlace','PuAddress',
     'PuSuburb','PuLat','PuLong','TargetVariable']]

In [9]:
val_features = val_data[['BookingSource','Conditions',
     'HasCondition','IsAccountBooking','CreatedDate','RequestedDate',
     'BookingFleet','DispatchFleet','AreaNumber','PuPlace','PuAddress',
     'PuSuburb','PuLat','PuLong','TargetVariable']]

In [10]:
train_features.head()

,BookingSource,Conditions,HasCondition,IsAccountBooking,CreatedDate,RequestedDate,BookingFleet,DispatchFleet,AreaNumber,PuPlace,PuAddress,PuSuburb,PuLat,PuLong,TargetVariable
0,TelOp,0,No,No,2019-04-11 10:36:55,2019-04-11 11:00:00,13cabs Sydney,13cabs Sydney,177,House,11 Stewart St,Randwick,-33.921022,151.244202,Completed
1,TelOp,0,No,No,2019-06-23 09:52:55,2019-06-23 09:52:55,13cabs Sydney,13cabs Sydney,111,,145 Alfred Rd,Narraweena,-33.755931,151.273028,Cancelled
2,Internet,0,No,No,2019-12-01 00:14:02,2019-12-01 00:14:02,13cabs Sydney,13cabs Sydney,92,,284 Birrell Street,Bondi,-33.897201,151.261205,Completed
3,StratumIVR,0,No,No,2019-09-12 09:07:03,2019-09-12 09:07:04,13cabs Sydney,13cabs Sydney,182,,127-131 Cook Rd,Centennial Park,-33.895716,151.228609,Completed
4,Internet,0,No,No,2019-02-21 21:37:04,2019-02-21 21:37:04,13cabs Sydney,13cabs Sydney,4,,Unit 87/5010 Macquarie St,Sydney,-33.859737,151.213307,Cancelled


In [11]:
val_features.head()

,BookingSource,Conditions,HasCondition,IsAccountBooking,CreatedDate,RequestedDate,BookingFleet,DispatchFleet,AreaNumber,PuPlace,PuAddress,PuSuburb,PuLat,PuLong,TargetVariable
0,TelOp,0,No,No,2019-04-11 10:36:55,2019-04-11 11:00:00,13cabs Sydney,13cabs Sydney,177,House,11 Stewart St,Randwick,-33.921022,151.244202,Completed
1,TelOp,0,No,No,2019-06-23 09:52:55,2019-06-23 09:52:55,13cabs Sydney,13cabs Sydney,111,,145 Alfred Rd,Narraweena,-33.755931,151.273028,Cancelled
2,Internet,0,No,No,2019-12-01 00:14:02,2019-12-01 00:14:02,13cabs Sydney,13cabs Sydney,92,,284 Birrell Street,Bondi,-33.897201,151.261205,Completed
3,StratumIVR,0,No,No,2019-09-12 09:07:03,2019-09-12 09:07:04,13cabs Sydney,13cabs Sydney,182,,127-131 Cook Rd,Centennial Park,-33.895716,151.228609,Completed
4,Internet,0,No,No,2019-02-21 21:37:04,2019-02-21 21:37:04,13cabs Sydney,13cabs Sydney,4,,Unit 87/5010 Macquarie St,Sydney,-33.859737,151.213307,Cancelled


In [22]:
#concatenate training data and validation data for feature engineering
features = pd.concat([train_features, val_features], axis=0).reset_index(drop=True)

In [24]:
# check datatypes
features.dtypes

BookingSource        object
Conditions            int64
HasCondition         object
IsAccountBooking     object
CreatedDate          object
RequestedDate        object
BookingFleet         object
DispatchFleet        object
AreaNumber            int32
PuPlace              object
PuAddress            object
PuSuburb             object
PuLat               float64
PuLong              float64
TargetVariable       object
dtype: object

In [25]:
#BookingSource variable
Conditions_count = data['Conditions'].value_counts().rename_axis('unique_values').reset_index(name='counts')
Conditions_count

,unique_values,counts
0,0,2240288
1,4194304,304956
2,64,101869
3,262144,88487
4,16777216,38880
...,...,...
375,537133056,1
376,604012544,1
377,17196908544,1
378,262152,1


In [ ]:
#save the concatenated data
features_parsed = features.to_csv('feature1_parsed.csv')

## 2. Feature engineering

In [69]:
feature1 = features

In [ ]:
%%time
#feature_data_datatable = dt.fread('feature1_parsed.csv') 
#feature1 = feature_data_datatable.to_pandas()

### 2.1 New time features: 
booking by day, request by day, booking by hour, request by hour, create part of day, request part of day

In [70]:
# Changing the pickup_datetime and dropoff_datetime from object to datetime datatype
feature1['CreatedDate']=pd.to_datetime(feature1['CreatedDate'])
feature1['RequestedDate']=pd.to_datetime(feature1['RequestedDate'])

In [71]:
# Creating features based on day
feature1['create_by_day'] = feature1['CreatedDate'].dt.day_name()
feature1['request_by_day'] = feature1['RequestedDate'].dt.day_name()

In [72]:
# Creating features based on Hour
feature1['create_by_hour'] = feature1['CreatedDate'].dt.hour
feature1['request_by_hour'] = feature1['RequestedDate'].dt.hour
#Morning, which starts at 6:01 am and ends at 12:oopm
#Afternoon, which starts at 12:01 pm and ends at 18:00pm
#Evening, which starts at 18:01 and ends at 21:00pm
#Night, which start at 21:01 and ends at 6:00am

In [73]:
def part_of_day (t):
    if t in range (6,12):
        return "Morning"
    elif t in range (12,18):
        return "Afternoon"
    elif t in range (18,21):
        return "Evening"
    else:
        return "Night"

In [74]:
feature1['create_part_of_day']=feature1['create_by_hour'].apply(part_of_day)
feature1['request_part_of_day']=feature1['request_by_hour'].apply(part_of_day)

### 2.2 New feature: if booking fleet matches with the dispatch fleet(binary)

In [75]:
%%time
equal = list()
for i in range(feature1.shape[0]):
    if feature1['BookingFleet'][i] == feature1['DispatchFleet'][i]:
        equal.append('True')
    else:
        equal.append('False')

Wall time: 1min 29s


In [76]:
feature1['BookingisDispatch'] = equal

### 2.3 New feature: if the records of pick up surburbs exceed 10 (binary)

In [77]:
pick = feature1['PuSuburb'].value_counts().tail(1000).keys().tolist() #<10

In [78]:
feature1['pick'] = np.where(feature1['PuSuburb'].isin(pick), True, False)

### 2.4 Drop columns

In [80]:
feature1 = feature1.drop(columns=['CreatedDate', 'RequestedDate']) #date features

MemoryError: Unable to allocate 641. MiB for an array with shape (14, 6000000) and data type object

In [ ]:
feature1 = feature1.drop(columns=['PuSuburb','PuPlace','PuAddress']) # pick-up location related features

### 2.5 Encode categorical features

In [ ]:
feature1 = pd.get_dummies(feature1).reset_index(drop=True)
feature1.shape

## Modeling